In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "yahoo"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-4,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

L = 5
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
19.65764770844837
prediction for base model
0.6076920935476612


In [2]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.03, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_DR] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])

# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])


# baselines
# for mlp
mlp_acc = []
for repeat in np.arange(10):
    mlp = MLP(num_feature, batch_size=128)
    mlp.to(device)
    mlp.fit(x_train, y_train_mask, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp, x_test, y_test, feature_test)

    print(auc)

    mlp_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_acc = np.array(mlp_acc)
mlp_mean = mlp_acc.mean(0)
mlp_sd = mlp_acc.std(0)

print("[MLP] test auc:", mlp_mean[1], ' sd: ', mlp_sd[1])

# for exponential
mlp_exp_acc = []
for repeat in np.arange(10):
    mlp_exp = MLP_exp(num_feature, batch_size=128)
    mlp_exp.to(device)
    mlp_exp.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_exp.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_exp, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_exp, x_test, y_test, feature_test)

    print(auc)

    mlp_exp_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_exp_acc = np.array(mlp_exp_acc)
mlp_exp_mean = mlp_exp_acc.mean(0)
mlp_exp_sd = mlp_exp_acc.std(0)

print("[MLP_exp] test auc:", mlp_exp_mean[1], ' sd: ', mlp_exp_sd[1])

# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])

# for lognormalonential
mlp_lognormal_acc = []
for repeat in np.arange(10):
    mlp_lognormal = MLP_lognormal(num_feature, batch_size=128)
    mlp_lognormal.to(device)
    mlp_lognormal.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_lognormal.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_lognormal, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_lognormal, x_test, y_test, feature_test)

    print(auc)

    mlp_lognormal_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_lognormal_acc = np.array(mlp_lognormal_acc)
mlp_lognormal_mean = mlp_lognormal_acc.mean(0)
mlp_lognormal_sd = mlp_lognormal_acc.std(0)

print("[MLP_lognormal] test auc:", mlp_lognormal_mean[1], ' sd: ', mlp_lognormal_sd[1])

# for MF
mf_acc = []
for repeat in np.arange(10):
    mf = MF(num_user, num_item, batch_size=128)
    mf.to(device)
    mf.fit(x_train, y_train_mask, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf, x_test, y_test)
    recall_res = recall_func(mf, x_test, y_test)

    print(auc)

    mf_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_acc = np.array(mf_acc)
mf_mean = mf_acc.mean(0)
mf_sd = mf_acc.std(0)

print("[MF] test auc:", mf_mean[1], ' sd: ', mf_sd[1])

# for MF_IPS
mf_ips_acc = []
for repeat in np.arange(10):
    mf_ips = MF_IPS(num_user, num_item, batch_size=128)
    mf_ips.to(device)
    mf_ips.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=8e-4,tol=1e-5)
    test_pred = mf_ips.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_ips, x_test, y_test)
    recall_res = recall_func(mf_ips, x_test, y_test)

    print(auc)

    mf_ips_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_acc = np.array(mf_ips_acc)
mf_ips_mean = mf_ips_acc.mean(0)
mf_ips_sd = mf_ips_acc.std(0)

print("[MF_IPS] test auc:", mf_ips_mean[1], ' sd: ', mf_ips_sd[1])

# for MF_ASIPS
mf_asips_acc = []
for repeat in np.arange(10):
    mf_asips = MF_ASIPS(num_user, num_item, batch_size=128)
    mf_asips.to(device)
    mf_asips.fit(x_train, y_train_mask, y_ips, tao=1e-2, lr=0.05, lamb=2e-4,tol=1e-5, batch_size=128,stop=5)
    test_pred = mf_asips.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_asips, x_test, y_test)
    recall_res = recall_func(mf_asips, x_test, y_test)

    print(auc)

    mf_asips_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_asips_acc = np.array(mf_asips_acc)
mf_asips_mean = mf_asips_acc.mean(0)
mf_asips_sd = mf_asips_acc.std(0)

print("[MF_ASIPS] test auc:", mf_asips_mean[1], ' sd: ', mf_asips_sd[1])

# for MF_SNIPS
mf_snips_acc = []
for repeat in np.arange(10):
    mf_snips = MF_SNIPS(num_user, num_item, batch_size=128)
    mf_snips.to(device)
    mf_snips.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_snips.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_snips, x_test, y_test)
    recall_res = recall_func(mf_snips, x_test, y_test)

    print(auc)

    mf_snips_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_snips_acc = np.array(mf_snips_acc)
mf_snips_mean = mf_snips_acc.mean(0)
mf_snips_sd = mf_snips_acc.std(0)

print("[MF_SNIPS] test auc:", mf_snips_mean[1], ' sd: ', mf_snips_sd[1])

# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])

# for DR
mf_dr_jl_acc = []
for repeat in np.arange(10):
    mf_dr_jl = MF_DR_JL(num_user, num_item, batch_size=128)
    mf_dr_jl.to(device)
    mf_dr_jl.fit(x_train, y_train_mask, y_ips, lr=0.03, lamb=2e-3,tol=1e-5)
    test_pred = mf_dr_jl.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_jl, x_test, y_test)
    recall_res = recall_func(mf_dr_jl, x_test, y_test)

    print(auc)

    mf_dr_jl_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_jl_acc = np.array(mf_dr_jl_acc)
mf_dr_jl_mean = mf_dr_jl_acc.mean(0)
mf_dr_jl_sd = mf_dr_jl_acc.std(0)

print("[MF_DR_JL] test auc:", mf_dr_jl_mean[1], ' sd: ', mf_dr_jl_sd[1])

# for DR
mf_mrdr_acc = []
for repeat in np.arange(10):
    mf_mrdr = MF_MRDR_JL(num_user, num_item, batch_size=128)
    mf_mrdr.to(device)
    mf_mrdr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=4e-3,tol=1e-5)
    test_pred = mf_mrdr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_mrdr, x_test, y_test)
    recall_res = recall_func(mf_mrdr, x_test, y_test)

    print(auc)

    mf_mrdr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_mrdr_acc = np.array(mf_mrdr_acc)
mf_mrdr_mean = mf_mrdr_acc.mean(0)
mf_mrdr_sd = mf_mrdr_acc.std(0)

print("[MF_MRDR_JL] test auc:", mf_mrdr_mean[1], ' sd: ', mf_mrdr_sd[1])

[MF-IPS-DF] epoch:27, xent:550.6366682052612
0.6980636178536438
[MF-IPS-DF] epoch:34, xent:551.3488283157349
0.6886423092597276
[MF-IPS-DF] epoch:31, xent:549.3030214309692
0.7018749241199265
[MF-IPS-DF] epoch:28, xent:550.369574546814
0.7032471451693101
[MF-IPS-DF] epoch:28, xent:549.3523960113525
0.695734921227784
[MF-IPS-DF] epoch:32, xent:548.1185102462769
0.6964415242054534
[MF-IPS-DF] epoch:28, xent:549.312554359436
0.6982709589462704
[MF-IPS-DF] epoch:25, xent:549.1200771331787
0.699902428858323
[MF-IPS-DF] epoch:28, xent:549.515588760376
0.699001437564909
[MF-IPS-DF] epoch:25, xent:549.7543172836304
0.7021648149995476
[MF_DR] test auc: 0.6983344082204896  sd:  [0.00042065 0.00398247 0.00822492 0.006177   0.00829775 0.00751697]
[MF-DR-JL-DF] epoch:22, xent:52031211.4375
0.7178386416519547
[MF-DR-JL-DF] epoch:19, xent:51906182.875
0.7177552219023338
[MF-DR-JL-DF] epoch:17, xent:52105189.5
0.7165410440634137
[MF-DR-JL-DF] epoch:25, xent:51905860.4375
0.72003819715905
[MF-DR-JL-DF]

In [3]:
np.vstack([mlp_mean, mlp_exp_mean, mlp_weibull_mean, mlp_lognormal_mean, mf_mean, mf_ips_mean, mf_asips_mean, mf_snips_mean, mf_dr_mean, mf_dr_jl_mean, mf_mrdr_mean, mf_ips_df_mean, mf_dr_df_mean ])

array([[0.24396525, 0.61675712, 0.40961975, 0.50528685, 0.26055605,
        0.55066244],
       [0.24732247, 0.63222197, 0.52681738, 0.60518774, 0.36105108,
        0.64505322],
       [0.23811164, 0.63262977, 0.52580577, 0.60530644, 0.36000601,
        0.64532908],
       [0.25308568, 0.63128853, 0.52567277, 0.60530652, 0.35973184,
        0.64443253],
       [0.25387342, 0.65184501, 0.57376555, 0.64961399, 0.40492333,
        0.68828698],
       [0.27166919, 0.64181604, 0.56129259, 0.63975343, 0.39386607,
        0.68372929],
       [0.26381833, 0.65617046, 0.56843215, 0.64591967, 0.39965758,
        0.68497128],
       [0.25367841, 0.65035522, 0.57682484, 0.65242919, 0.40757221,
        0.69051838],
       [0.23570256, 0.6892964 , 0.57711028, 0.6547865 , 0.39104827,
        0.6862847 ],
       [0.26876797, 0.65827969, 0.58037185, 0.6557002 , 0.40692478,
        0.69030438],
       [0.27445041, 0.67832711, 0.59629052, 0.6661554 , 0.41441251,
        0.69178859],
       [0.23415659, 0

In [5]:
binres = np.vstack([mlp_mean, mlp_exp_mean, mlp_weibull_mean, mlp_lognormal_mean, mf_mean, mf_ips_mean, mf_asips_mean, mf_snips_mean, mf_dr_mean, mf_dr_jl_mean, mf_mrdr_mean, mf_ips_df_mean, mf_dr_df_mean ])

binnres = pd.DataFrame(binres, index=['MLP', 'Expoential', 'Weibull', 'Lognormal', 'MF', 'MF-IPS', 'MF-ASIPS', 'MF-SNIPS', 'MF-DR', 'MF-DR-JL', 'MF-MRDR', 'MF-IPS-DeF', 'MF-DR-JL-DeF'], columns=['MSE', 'AUC', 'ndcg5', 'ndcg10', 'recall5', 'recall10'])
binnres.to_csv( "result_"+dataset_name+".csv" )

binsd = np.vstack([mlp_sd, mlp_exp_sd, mlp_weibull_sd, mlp_lognormal_sd, mf_sd, mf_ips_sd, mf_asips_sd, mf_snips_sd, mf_dr_sd, mf_dr_jl_sd, mf_mrdr_sd, mf_ips_df_sd, mf_dr_df_sd ])

binsd = pd.DataFrame(binsd, index=['MLP', 'Expoential', 'Weibull', 'Lognormal', 'MF', 'MF-IPS', 'MF-ASIPS', 'MF-SNIPS', 'MF-DR', 'MF-DR-JL', 'MF-MRDR', 'MF-IPS-DeF', 'MF-DR-JL-DeF'], columns=['MSE', 'AUC', 'ndcg5', 'ndcg10', 'recall5', 'recall10'])
binsd.to_csv( "result_"+dataset_name+"_sd.csv" )